Notebook dedicato all'estrazione di queste informazioni:
- tables
- tables caption
- footnotes in the tables or in the captions
- paragraphs containing references to the extracted tables

In [20]:
from lxml import html
import os
import json
import re

In [39]:
# query xpath che serviranno per estrarre le informazioni
querys = {
    "figures": "//figure[contains(@class,'ltx_table')]",
    "caption": ".//figcaption//text()",
    "html_table": "./div | ./table | ./p",
    "table_id": "./@id",
    "footnotes": ".//figcaption[contains(text(), 'Note')]/text()",
    "footnotes_bli": ".//cite//a/text() | .//cite//span/text()",
    "paragraphs": "//div[contains(@class, 'ltx_para')]//p | //div[contains(@class, 'ltx_para ltx_noindent')]//p"
}

In [40]:

# Funzione per sostituire caratteri speciali
def pulisci_stringa(s:str):
    s = s.replace('\u00a0', ' ')  # Sostituisce lo spazio non interrompibile con spazio normale
    s = s.replace('\u201c', '"')  # Sostituisce virgolette di apertura con virgolette standard
    s = s.replace('\u201d', '"')  # Sostituisce virgolette di chiusura con virgolette standard
    s = s.replace('\u00fc', 'ü')  # Sostituisce ü con u con umlaut
    s = s.replace('\u00e9', 'é')  # Sostituisce é con e accentata
    s = s.replace('\u00e8', 'è')  # Sostituisce è con e accentata
    s = s.replace('\u00e0', 'à')  # Sostituisce à con a accentata
    s = s.replace('\u00f3', 'ó')  # Sostituisce ó con o accentata
    s = s.replace('\u00f4', 'ô')  # Sostituisce ô con o circonflesso
    s = s.replace('\u00e2', 'â')  # Sostituisce â con a circonflesso
    s = s.replace('\u00ea', 'ê')  # Sostituisce ê con e circonflesso
    s = s.replace('\u00ee', 'î')  # Sostituisce î con i circonflesso
    s = s.replace('\u00fb', 'û')  # Sostituisce û con u circonflesso
    s = s.replace('\u00e7', 'ç')  # Sostituisce ç con c cediglia
    s = s.replace('\u00f9', 'ù')  # Sostituisce ù con u accentata
    s = s.replace('\u00f1', 'ñ')  # Sostituisce ñ con n tilde
    s = s.replace('\u00f6', 'ö')  # Sostituisce ö con o umlaut
    s = s.replace('\u00e4', 'ä')  # Sostituisce ä con a umlaut
    s = s.replace('\u00eb', 'ë')  # Sostituisce ë con e umlaut
    s = s.replace('\u00ef', 'ï')  # Sostituisce ï con i umlaut
    s = s.replace('\u00fc', 'ü')  # Sostituisce ü con u umlaut
    s = s.replace('\u00ff', 'ÿ')  # Sostituisce ÿ con y umlaut
    s = s.replace('\u00e5', 'å')  # Sostituisce å con a ring
    s = s.replace('\u00f8', 'ø')  # Sostituisce ø con o barrata
    s = s.replace('\u00e6', 'æ')  # Sostituisce æ con ae legato
    s = s.replace('\u00f6', 'ö')  # Sostituisce ö con o umlaut
    s = s.replace('\u00e4', 'ä')  # Sostituisce ä con a umlaut
    s = s.replace('\u00eb', 'ë')  # Sostituisce ë con e umlaut
    s = s.replace('\u00ef', 'ï')  # Sostituisce ï con i umlaut
    s = s.replace('\u00fc', 'ü')  # Sostituisce ü con u umlaut
    s = s.replace('\u00ff', 'ÿ')  # Sostituisce ÿ con y umlaut
    s = s.replace('\u00fc', 'ü')  # Sostituisce ü con u umlaut
    s = s.replace('\u00a0', ' ')  
    return s

# Funzione per estrarre il numero dalla stringa, sia arabo che romano
def extract_table_number(escaped_caption):
    # Prova prima con un numero arabo
    arabic_number = re.search(r'\d+', escaped_caption)
    if arabic_number:
        return arabic_number.group()  # Restituisce il numero arabo come stringa
    
    # Se non c'è numero arabo, prova con numero romano
    roman_number = re.search(r'\b[I|V|X|L|C|D|M]+\b', escaped_caption)
    if roman_number:
        return roman_number.group()

    return None  # Nessun numero trovato

In [41]:
def get_footnotesBlib(tree:html.HtmlElement, figure: html.HtmlElement):
    cite_numbers = figure.xpath(querys["footnotes_bli"])
    footnotes = set()

    for cite in cite_numbers:
        cite_number = cite.strip('[]')

        footnote = tree.xpath(f'//li[@id="bib.bib{cite_number}"]/span[@class="ltx_bibblock"]//text()')
        if footnote:
            full_text = " ".join(footnote).strip()
            full_text = pulisci_stringa(full_text)
            footnotes.add(full_text)
            
    footnotes = list(footnotes)
    return footnotes

def get_footnotes(tree:html.HtmlElement, figure: html.HtmlElement):
    footnotes = figure.xpath(querys["footnotes"])
    
    return footnotes

def get_references(tree: html.HtmlElement, caption):
    references = []
    escaped_caption = json.dumps(caption.split(":")[0])
    number = extract_table_number(escaped_caption)
    xpath_query = (
        f'//div[contains(@class, "ltx_para")]//p[a[contains(@title, {escaped_caption})]] | '
        f'//div[contains(@class, "ltx_para")]//p[a[span[text()="{number}"]]] | '
        f'//div[contains(@class, "ltx_para")]//p[span[text()="{number}"]] | '
        f'//div[contains(@class, "ltx_para")]//p[contains(text(), "{number}")] | '
        f'//div[contains(@class, "ltx_para")]//p[span[@class="ltx_text ltx_ref_tag" and text()="{number}"]] | '
        f'//div[contains(@class, "ltx_para")]//p[span[contains(text(), "{number}")]] |'
        f'//div[contains(@class, "ltx_para ltx_noindent")]//p[a[contains(@title, {escaped_caption})]] | '
        f'//div[contains(@class, "ltx_para ltx_noindent")]//p[a[span[text()="{number}"]]] | '
        f'//div[contains(@class, "ltx_para ltx_noindent")]//p[span[text()="{number}"]] | '
        f'//div[contains(@class, "ltx_para ltx_noindent")]//p[contains(text(), "{number}")] | '
        f'//div[contains(@class, "ltx_para ltx_noindent")]//p[span[@class="ltx_text ltx_ref_tag" and text()="{number}"]] | '
        f'//div[contains(@class, "ltx_para ltx_noindent")]//p[span[contains(text(), "{number}")]]'
    )
    try:
        relevant_paragraphs = tree.xpath(xpath_query)
        
    except Exception as e:
        print(f"Errore nel calcolo dell'XPath: {e}")
        relevant_paragraphs = []
    
    for paragraph in relevant_paragraphs:
        references.append(html.tostring(paragraph, encoding="utf-8").decode("utf-8"))
        
    #Procedo a trasformare i pezzi di codice html di references in stringhe con solo il contenuto testuale
    for i in range(len(references)):
        references[i] = re.sub(r'<.*?>', '', references[i])
    
    #pulisco le references
    references = [pulisci_stringa(reference) for reference in references]

    # Trasforma i pezzi di codice HTML in stringhe con solo il contenuto testuale
    filtered_references = []
    for reference in references:
        # Rimuovi i tag HTML
        text = re.sub(r'<.*?>', '', reference).strip()
        # Controlla se contiene "Table" e il numero specificato
        if f"Table {number} " in text:
            filtered_references.append(text)
            
        if f"Table {number}," in text:
            filtered_references.append(text)
        
        if f"Table {number}." in text:
            filtered_references.append(text)
        
        if f"Table {number}:" in text:
            filtered_references.append(text)
        
        if f"Table {number})" in text:
            filtered_references.append(text)
        
        if f"Table {number}(" in text:
            filtered_references.append(text)
        
        if f"Table {number}]" in text:
            filtered_references.append(text)
        
        if f"Table {number}[" in text:
            filtered_references.append(text)
        
        if f"table {number} " in text:
            filtered_references.append(text)
        
        if f"table {number}," in text:
            filtered_references.append(text)
        
        if f"table {number}." in text:
            filtered_references.append(text)
        
        if f"table {number}:" in text:
            filtered_references.append(text)
        
        if f"table {number})" in text:
            filtered_references.append(text)
        
        if f"table {number}(" in text:
            filtered_references.append(text)
        
        if f"table {number}]" in text:
            filtered_references.append(text)
        
        if f"table {number}[" in text:
            filtered_references.append(text)

    return filtered_references

In [42]:
# Funzione per estrarre le informazioni da un file HTML
def extract_info_from_html(paper_path):
    result = {}

    with open(paper_path, "r", encoding="utf-8") as paper:
        content = paper.read()

    tree: html.HtmlElement = html.fromstring(content)
    figures: list[html.HtmlElement] = tree.xpath(querys["figures"])
    
    for index, figure in enumerate(figures):
        # estrazione table id
        table_id = figure.xpath(querys["table_id"])[0]

        # estrazione html_table
        html_table = figure.xpath(querys["html_table"])

        '''
        if len(html_table) == 0:
            print(index,paper_path)
        '''
        
        # estrazione caption
        caption = figure.xpath(querys["caption"])
        if type(caption) == list:
            caption = " ".join(caption)
        
        # Crea l'oggetto risultato per la tabella
        result[table_id] = {
            "caption": caption,
            "table": html.tostring(figure, encoding="utf-8").decode("utf-8"),
            "footnotes": get_footnotes(tree, figure),  # Aggiungi i piè di pagina trovati
            "references": get_references(tree, caption)
        }

    return result

In [44]:
papers = os.listdir("sources")
papers.remove(".txt")
papers.sort()
#Se non esiste la cartella extraction, creo la cartella
if not os.path.exists("extraction"):
    os.makedirs("extraction")

i=1
for paper in papers:
        print(f"Paper elaborati:  {i}, actual paper: {paper}")
        i=i+1
    #if(paper == "2406.05784.html" or paper == "2403.07938.html"):
        print(f"Elaborazione del paper {paper}")
        result = extract_info_from_html(f"sources/{paper}")
        paper_name_split = paper.split(".")
        paper_name = f"{paper_name_split[0]}.{paper_name_split[1]}"
        
        with open(f"extraction/{paper_name}.json","w") as file:
            json.dump(result, file, indent=4)

Paper elaborati:  1, actual paper: 2402.13004.html
Elaborazione del paper 2402.13004.html
Paper elaborati:  2, actual paper: 2402.13152.html
Elaborazione del paper 2402.13152.html
Paper elaborati:  3, actual paper: 2402.13208.html
Elaborazione del paper 2402.13208.html
Paper elaborati:  4, actual paper: 2402.13432.html
Elaborazione del paper 2402.13432.html
Paper elaborati:  5, actual paper: 2402.14185.html
Elaborazione del paper 2402.14185.html
Paper elaborati:  6, actual paper: 2402.15151.html
Elaborazione del paper 2402.15151.html
Paper elaborati:  7, actual paper: 2402.16124.html
Elaborazione del paper 2402.16124.html
Paper elaborati:  8, actual paper: 2402.17184.html
Elaborazione del paper 2402.17184.html
Paper elaborati:  9, actual paper: 2402.17249.html
Elaborazione del paper 2402.17249.html
Paper elaborati:  10, actual paper: 2402.17496.html
Elaborazione del paper 2402.17496.html
Paper elaborati:  11, actual paper: 2402.17723.html
Elaborazione del paper 2402.17723.html
Paper el